In [10]:
import os
import pdfplumber
import pandas as pd
from gpt_bot_driver import gpt_bot_driver
import time
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
pd.set_option('display.max_colwidth', None)

data_dir = '../data/temp'
fnames = os.listdir(data_dir)

# gpt 챗봇 실행
bot = gpt_bot_driver("haesung.kim")

for fname in fnames:
    if fname.endswith('.pdf'):        
        print('processing', fname)
        # 텍스트 파일을 작성할 경로와 파일명 지정
        input_path = os.path.join(data_dir, fname)
        output_path = os.path.join(data_dir, fname).rstrip('.pdf') +'.docx'

        doc = Document()

        pdf = pdfplumber.open(input_path)            
        for page_index, page in enumerate(pdf.pages):
            # 텍스트 추출
            page_text = page.extract_text()
            table = page_text.maketrans('\n',' ') # page \n 제거
            page_text_processed = page_text.translate(table)
            
            # 테이블 추출
            tables = page.extract_tables(
                table_settings = {
                    "vertical_strategy": "lines", 
                    "horizontal_strategy": "lines",
                }
            )
            
            # Raw Text 출력
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'Raw Text', level=1)
            doc.add_paragraph(page_text)
            doc.add_page_break()
            
            # 영어 번역 출력
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'English Text', level=1)
            bot.delete_conversation_history() # 대화 이력 삭제
            time.sleep(3)
            question = ' Please translate the previous text into English and respond only in English.'
            answer1 = bot.get_gpt_answer(page_text_processed + ' ' + question) # 질의 진행
            doc.add_paragraph(answer1)
            doc.add_page_break()
            
            # 한국어 번역 출력
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'Korean Text', level=1)
            bot.delete_conversation_history() # 대화 이력 삭제
            time.sleep(3)
            question = ' 앞의 글을 한국어로 번역해줘, 한국어로만 답변해줘'
            answer2 = bot.get_gpt_answer(page_text_processed + ' ' + question) # 질의 진행
            doc.add_paragraph(answer2)
            doc.add_page_break()
            
            # answer1 \n 제거
            table = answer1.maketrans('\n',' ')
            answer1_processed = answer1.translate(table)
            
            tables = page.extract_tables(
                table_settings = {
                    "vertical_strategy": "lines", 
                    "horizontal_strategy": "lines",
                }
            )

            # 영어 요약 출력
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'Summarization', level=1)
            bot.delete_conversation_history() # 대화 이력 삭제
            time.sleep(3)
            question = ' Please summarize the previous text and respond only in English.'
            answer3 = bot.get_gpt_answer(answer1_processed + ' ' + question) # 질의 진행
            doc.add_paragraph(answer3)
            doc.add_page_break()
            
            # 영어 키워드 추출 출력
            doc.add_heading(f'Page {page_index+1}', level=0)
            doc.add_heading(f'Extracted Keyword', level=1)
            bot.delete_conversation_history() # 대화 이력 삭제
            time.sleep(3)
            question = ' Please extract important keywords from the previous text and respond only in English.'
            answer4 = bot.get_gpt_answer(answer1_processed + ' ' + question) # 질의 진행
            doc.add_paragraph(answer4)
            doc.add_page_break()
            
            # Raw Table 출력
            if len(tables):
                doc.add_heading(f'Page {page_index+1}', level=0)
                doc.add_heading(f'Raw Table', level=1)    
                for table_index, table in enumerate(tables):
                    doc.add_heading(f'Table {page_index+1}-{table_index+1}', level=2)
                    
                    # Convert list of lists to dataframe
                    df = pd.DataFrame(table)
                    # add a table to the end and create a reference variable
                    # extra row is so we can add the header row
                    t = doc.add_table(df.shape[0]+1, df.shape[1])

                    for j in range(df.shape[-1]):
                        t.cell(0,j).text = str(df.columns[j])

                    for i in range(df.shape[0]):
                        for j in range(df.shape[-1]):
                            t.cell(i+1,j).text = str(df.values[i,j])
                    # print(f'-----------------------------------------------------English Table-----------------------------------------------------')
                    # print()
                    # print(f'----------------------------------------------------- Korean Table-----------------------------------------------------')
                    # print()
                doc.add_page_break()
        doc.save(output_path)
    

processing Workstation monitor_24003PPT.pdf
processing 四川省肿瘤医院医用显示器招标文件.pdf


In [3]:
import os
import pdfplumber
import pandas as pd
from gpt_bot_driver import gpt_bot_driver
import time
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from pdfminer.high_level import extract_text

pd.set_option('display.max_colwidth', None)

data_dir = '../data/temp'
fnames = os.listdir(data_dir)

for fname in fnames:
    if fname.endswith('.pdf'):        
        print('processing', fname)
        
        # 분석 대상 파일 경로
        input_path = os.path.join(data_dir, fname)
        # 분석 결과 파일 경로
        # output_path = os.path.join(data_dir, fname).rstrip('.pdf') +'.docx'
        
        text = extract_text(input_path)

        pdf = pdfplumber.open(input_path)            
        for page_index, page in enumerate(pdf.pages):
            # 텍스트 추출
            page_text = page.extract_text()
            table = page_text.maketrans('\n',' ') # page \n 제거
            page_text_processed = page_text.translate(table)
            print(page_text)

processing 南充市中心医院医用显示器采购项目.pdf
竞竞 争争 性性 谈谈 判判 文文 件件
（（货货物物类类））
采采购购项项目目名名称称：：南南充充市市中中心心医医院院医医用用显显示示器器采采购购项项目目
采采购购项项目目编编号号：：N5113012022000247
南南充充市市中中心心医医院院（（川川北北医医学学院院附附属属南南充充市市中中心心医医院院））
南南充充市市政政府府采采购购中中心心共共同同编编制制
2022年年11月月07日日
第第一一章章 竞竞争争性性谈谈判判邀邀请请
南充市政府采购中心（以下简称“代理机构”）受南充市中心医院（川北医学院附属南充市中心医院）委托，拟对南充市中
心医院医用显示器采购项目采用竞争性谈判采购方式进行采购，兹邀请供应商参加本项目的竞争性谈判。
一一、、项项目目编编号号：：N5113012022000247
二二、、项项目目名名称称：：南南充充市市中中心心医医院院医医用用显显示示器器采采购购项项目目
三三、、谈谈判判项项目目简简介介：：
采购医用显示器一批
四四、、邀邀请请供供应应商商：：
本次采购采取公告征集邀请谈判的供应商。
公告征集：本次竞争性谈判邀请在四川政府采购网（www.ccgp-sichuan.gov.cn）上以公告形式发布，公告期限自公告
-第1页-
发布之日起3个工作日。
五五、、供供应应商商参参加加本本次次政政府府采采购购活活动动应应具具备备的的条条件件
（一）满足《中华人民共和国政府采购法》第二十二条规定；
（二）落实政府采购政策需满足的资格要求：
执行政府采购促进中小企业发展的相关政策：
采购包1（合同包一）：属于专门面向中小企业采购。
注：监狱企业和残疾人福利性单位视同小微企业，符合中小企业划分标准的个体工商户视同中小企业。
（三）本项目的特定资格要求：
采购包1：
无
六六、、电电子子化化采采购购相相关关事事项项
本项目实行电子化采购，使用的电子化交易系统为：四川省政府采购一体化平台（以下简称“采购一体化平台”）的项目电
子化交易系统（以下简称项目电子化交易系统），登录方式及地址：通过四川政府采购网（www.ccgp-sichuan.gov.cn）首
页供应商用户登录采购一体化平台，进入项目电子化交易系统。供应商应当按照以下要求，参与本次电子化采购活动。
（一）供应商应当自行在四川

In [7]:
from langchain_community.document_loaders import PyPDFLoader

# 1. Loader
loader = PyPDFLoader('D:\projects\langchain\data\\temp\南充市中心医院医用显示器采购项目.pdf')
pages = loader.load_and_split() # page split
pages

[Document(page_content='竞\n竞\n \n争\n争\n \n性\n性\n \n谈\n谈\n \n判\n判\n \n文\n文\n \n件\n件\n（货物类）\n（货物类）\n采购项目名称：\n采购项目名称：\n南充市中心医院医用显示器采购项目\n南充市中心医院医用显示器采购项目\n采购项目编号：\n采购项目编号：\nN5113012022000247\n南充市中心医院（川北医学院附属南充市中心医院）\n南充市中心医院（川北医学院附属南充市中心医院）\n南充市政府采购中心\n南充市政府采购中心\n共同编制\n共同编制\n2022\n年\n年\n11\n月\n月\n07\n日\n日\n第一章\n第一章\n \n竞争性谈判邀请\n竞争性谈判邀请\n南充市政府采购中心\n（以下简称\n“\n代理机构\n”\n）受\n南充市中心医院（川北医学院附属南充市中心医院）\n委托，拟对\n南充市中\n心医院医用显示器采购项目\n采用竞争性谈判采购方式进行采购，兹邀请供应商参加本项目的竞争性谈判。\n一、项目编号：\n一、项目编号：\nN5113012022000247\n二、项目名称：\n二、项目名称：\n南充市中心医院医用显示器采购项目\n南充市中心医院医用显示器采购项目\n三、谈判项目简介：\n三、谈判项目简介：\n采购医用显示器一批\n四、邀请供应商：\n四、邀请供应商：\n本次采购采取公告征集邀请谈判的供应商。\n公告征集：本次竞争性谈判邀请在四川政府采购网（\nwww.ccgp-sichuan.gov.cn\n）上以公告形式发布，公告期限自公告\n-\n第\n1\n页\n-', metadata={'source': 'D:\\projects\\langchain\\data\\temp\\南充市中心医院医用显示器采购项目.pdf', 'page': 0}),
 Document(page_content='发布之日起\n3\n个工作日。\n五、供应商参加本次政府采购活动应具备的条件\n五、供应商参加本次政府采购活动应具备的条件\n（一）满足《中华人民共和国政府采购法》第二十二条规定；\n（二）落实政府采购政策需满足的资格要求：\n执行政府采购促进中小企业发展的相关政策：\n采购包\n1\n（合同包一）：属于专门面向中小企业采购。\n注：监狱企业和残

In [12]:
import fitz  # import PyMuPDF

doc = fitz.open('D:\projects\langchain\data\\temp\南充市中心医院医用显示器采购项目.pdf')
page = doc[0]  # example first page

# extract text including its coordinates
blocks = page.get_text("dict", sort=True, flags=fitz.TEXTFLAGS_TEXT)["blocks"]
old_bbox = fitz.EMPTY_RECT()  # store previous bbox here
old_text = ""  # store previous text here
for b in blocks:  # loop over text blocks
    for l in b["lines"]:  # lines in current block
        bbox = fitz.Rect(l["bbox"])  # line boundary box
        # text in line - remove leading trailing spaces where possible
        text = " ".join([s["text"].strip() for s in l["spans"]]).strip()
        # check if new bbox overlaps old bbox
        isect = abs(bbox & old_bbox) / abs(bbox)  # overlap ratio
        if text != old_text or isect < 0.5:  # text unequal or no overlap
            print(text)  # print text
        old_text = text  # store for next
        old_bbox = +bbox  # store for next

竞
竞  争
争  性
性  谈
谈  判
判  文
文  件
件
（货物类）
采购项目名称：
采购项目名称：南充市中心医院医用显示器采购项目
南充市中心医院医用显示器采购项目
采购项目编号：
采购项目编号： N5113012022000247
南充市中心医院（川北医学院附属南充市中心医院）
南充市政府采购中心
南充市政府采购中心共同编制
共同编制
2022 年
年 11 月
月 07 日
日
第一章
第一章  竞争性谈判邀请
竞争性谈判邀请
南充市政府采购中心（以下简称 “ 代理机构 ” ）受南充市中心医院（川北医学院附属南充市中心医院）委托，拟对南充市中
心医院医用显示器采购项目采用竞争性谈判采购方式进行采购，兹邀请供应商参加本项目的竞争性谈判。
一、项目编号：
一、项目编号： N5113012022000247
二、项目名称：
二、项目名称：南充市中心医院医用显示器采购项目
南充市中心医院医用显示器采购项目
三、谈判项目简介：
采购医用显示器一批
四、邀请供应商：
本次采购采取公告征集邀请谈判的供应商。
公告征集：本次竞争性谈判邀请在四川政府采购网（ www.ccgp-sichuan.gov.cn ）上以公告形式发布，公告期限自公告
- 第 1 页 -


In [ ]:
竞 争 性 谈 判 文 件
（货物类）
采购项目名称：南充市中心医院医用显示器采购项目
采购项目编号：N5113012022000247
南充市中心医院（川北医学院附属南充市中心医院）
南充市政府采购中心共同编制
2022年11月07日
第一章 竞争性谈判邀请
南充市政府采购中心（以下简称“代理机构”）受南充市中心医院（川北医学院附属南充市中心医院）委托，拟对南充市中
心医院医用显示器采购项目采用竞争性谈判采购方式进行采购，兹邀请供应商参加本项目的竞争性谈判。
一、项目编号：N5113012022000247
二、项目名称：南充市中心医院医用显示器采购项目
三、谈判项目简介：
采购医用显示器一批
四、邀请供应商：
本次采购采取公告征集邀请谈判的供应商。
公告征集：本次竞争性谈判邀请在四川政府采购网（www.ccgp-sichuan.gov.cn）上以公告形式发布，公告期限自公告
-第1页-